## Save

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/Noha-Magdy/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


  Cloning https://github.com/Noha-Magdy/segment-anything.git to /tmp/pip-req-build-fymjfdq7
  Running command git clone --filter=blob:none --quiet https://github.com/Noha-Magdy/segment-anything.git /tmp/pip-req-build-fymjfdq7
  Resolved https://github.com/Noha-Magdy/segment-anything.git to commit 98439a88601f4290b63152ab48b1efc88e4c6cd9
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36606 sha256=2e3dc72dcfaabd513ee4439681a137497e58eff132025d593ae1dcb28b1f5194
  Stored in directory: /tmp/pip-ephem-wheel-cache-nn8c1ef_/wheels/a5/4a/ea/b9ae92897766fc687a9f74ac9c0351adfd61e2df1e37cb2f16
Successfully built segment-anything
--2023-08-08 10:19:58--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (ra

In [ ]:

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
from torchsummary import summary
import cv2
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
import nibabel as nib
import torch.nn.functional as F
from skimage import transform, io, segmentation

# set seeds
torch.manual_seed(2023)
np.random.seed(2023)


# create_folders
base = "/content/"
os.makedirs(base + "BraTS_data", exist_ok=True)


# prepare SAM model
model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to('cuda')

In [ ]:
print(len(os.listdir("/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData")))

43


In [ ]:
os.makedirs(base + "BraTS_data/flair/z", exist_ok=True)
os.makedirs(base + "BraTS_data/flair/z/images", exist_ok=True)
os.makedirs(base + "BraTS_data/flair/z/embeddings", exist_ok=True)

src_folder = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/"
src_folder = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/"
image_size = 256

# files = os.listdir(src_folder)
files = ["BraTS-SSA-00002-000"]
for i in tqdm(files):
  img =  nib.load(src_folder + i + "/"+ i +"-t2f.nii.gz")
  img = img.get_fdata()
  num_slices = img.shape[2]


  for s in range(num_slices):
    image = img[:,:,s]
    path = base + "BraTS_data/flair/z/embeddings/" +i+ "_"+ str(s)+".npy"
    # if os.path.exists(path): continue
    # np.save(base + "BraTS_data_africa/flair/y/images/"+i+"_"+ str(s), image)
    # image = squarify(image)


    lower_bound, upper_bound = np.percentile(image, 0.5), np.percentile(image, 99.5)
    image_data_pre = np.clip(image, lower_bound, upper_bound)
    d = (np.max(image_data_pre)-np.min(image_data_pre))
    d = 1 if d==0 else d
    image_data_pre = ((image_data_pre - np.min(image_data_pre))/d) *255.0
    image_data_pre[image==0] = 0
    image = transform.resize(image_data_pre[:,:], (image_size, image_size), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
    image = np.stack((image, image, image), axis = -1 )
    image = np.uint8(image)

    sam_transform = ResizeLongestSide(sam_model.image_encoder.img_size)
    image = sam_transform.apply_image(image)
    # resized_shapes.append(resize_img.shape[:2])
    image = torch.as_tensor(image.transpose(2, 0, 1)).to('cuda')
    # model input: (1, 3, 1024, 1024)
    image = sam_model.preprocess(image[None,:,:,:]) # (1, 3, 1024, 1024)
    image = image[0,...] # (3, 1024, 1024)
    # plt.imshow(image[1,:,:].cpu(), cmap ="gray")
    # plt.show()
    with torch.no_grad():
      embedding = sam_model.image_encoder(image[None,...])[0].cpu().numpy()
      # print(embedding.shape)
    np.save(base + "BraTS_data/flair/z/embeddings/" +i+ "_"+ str(s), embedding)

100%|██████████| 1/1 [01:35<00:00, 95.27s/it]


In [ ]:
os.makedirs(base + "BraTS_data/t1/z", exist_ok=True)
os.makedirs(base + "BraTS_data/t1/z/images", exist_ok=True)
os.makedirs(base + "BraTS_data/t1/z/embeddings", exist_ok=True)

src_folder = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/"
image_size = 256

# files = os.listdir(src_folder)
files = ["BraTS-SSA-00002-000"]
for i in tqdm(files):
  img =  nib.load(src_folder + i + "/"+ i +"-t1n.nii.gz")
  img = img.get_fdata()
  num_slices = img.shape[2]


  for s in range(num_slices):
    image = img[:,:,s]
    path = base + "BraTS_data/t1/z/embeddings/" +i+ "_"+ str(s)+".npy"
    # if os.path.exists(path): continue
    # np.save(base + "BraTS_data_africa/flair/y/images/"+i+"_"+ str(s), image)
    # image = squarify(image)


    lower_bound, upper_bound = np.percentile(image, 0.5), np.percentile(image, 99.5)
    image_data_pre = np.clip(image, lower_bound, upper_bound)
    d = (np.max(image_data_pre)-np.min(image_data_pre))
    d = 1 if d==0 else d
    image_data_pre = ((image_data_pre - np.min(image_data_pre))/d) *255.0
    image_data_pre[image==0] = 0
    image = transform.resize(image_data_pre[:,:], (image_size, image_size), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
    image = np.stack((image, image, image), axis = -1 )
    image = np.uint8(image)

    sam_transform = ResizeLongestSide(sam_model.image_encoder.img_size)
    image = sam_transform.apply_image(image)
    # resized_shapes.append(resize_img.shape[:2])
    image = torch.as_tensor(image.transpose(2, 0, 1)).to('cuda')
    # model input: (1, 3, 1024, 1024)
    image = sam_model.preprocess(image[None,:,:,:]) # (1, 3, 1024, 1024)
    image = image[0,...] # (3, 1024, 1024)
    # plt.imshow(image[1,:,:].cpu(), cmap ="gray")
    # plt.show()
    with torch.no_grad():
      embedding = sam_model.image_encoder(image[None,...])[0].cpu().numpy()
      # print(embedding.shape)
    np.save(base + "BraTS_data/t1/z/embeddings/" +i+ "_"+ str(s), embedding)

  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [ ]:
os.makedirs(base + "BraTS_data/t1ce/z", exist_ok=True)
os.makedirs(base + "BraTS_data/t1ce/z/images", exist_ok=True)
os.makedirs(base + "BraTS_data/t1ce/z/embeddings", exist_ok=True)

src_folder = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/"
image_size = 256

# files = os.listdir(src_folder)
files = ["BraTS-SSA-00002-000"]
for i in tqdm(files):
  img =  nib.load(src_folder + i + "/"+ i +"-t1c.nii.gz")
  img = img.get_fdata()
  num_slices = img.shape[2]


  for s in range(num_slices):
    image = img[:,:,s]
    path = base + "BraTS_data/t1ce/z/embeddings/" +i+ "_"+ str(s)+".npy"
    # if os.path.exists(path): continue
    # np.save(base + "BraTS_data_africa/flair/y/images/"+i+"_"+ str(s), image)
    # image = squarify(image)


    lower_bound, upper_bound = np.percentile(image, 0.5), np.percentile(image, 99.5)
    image_data_pre = np.clip(image, lower_bound, upper_bound)
    d = (np.max(image_data_pre)-np.min(image_data_pre))
    d = 1 if d==0 else d
    image_data_pre = ((image_data_pre - np.min(image_data_pre))/d) *255.0
    image_data_pre[image==0] = 0
    image = transform.resize(image_data_pre[:,:], (image_size, image_size), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
    image = np.stack((image, image, image), axis = -1 )
    image = np.uint8(image)

    sam_transform = ResizeLongestSide(sam_model.image_encoder.img_size)
    image = sam_transform.apply_image(image)
    # resized_shapes.append(resize_img.shape[:2])
    image = torch.as_tensor(image.transpose(2, 0, 1)).to('cuda')
    # model input: (1, 3, 1024, 1024)
    image = sam_model.preprocess(image[None,:,:,:]) # (1, 3, 1024, 1024)
    image = image[0,...] # (3, 1024, 1024)
    # plt.imshow(image[1,:,:].cpu(), cmap ="gray")
    # plt.show()
    with torch.no_grad():
      embedding = sam_model.image_encoder(image[None,...])[0].cpu().numpy()
      # print(embedding.shape)
    np.save(base + "BraTS_data/t1ce/z/embeddings/" +i+ "_"+ str(s), embedding)

100%|██████████| 1/1 [05:09<00:00, 309.65s/it]


In [ ]:
os.makedirs(base + "BraTS_data/t2/z", exist_ok=True)
os.makedirs(base + "BraTS_data/t2/z/images", exist_ok=True)
os.makedirs(base + "BraTS_data/t2/z/embeddings", exist_ok=True)

src_folder = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/"
image_size = 256

# files = os.listdir(src_folder)
files = ["BraTS-SSA-00002-000"]
for i in tqdm(files):
  img =  nib.load(src_folder + i + "/"+ i +"-t2w.nii.gz")
  img = img.get_fdata()
  num_slices = img.shape[2]


  for s in range(num_slices):
    image = img[:,:,s]
    path = base + "BraTS_data/t2/z/embeddings/" +i+ "_"+ str(s)+".npy"
    # if os.path.exists(path): continue
    # np.save(base + "BraTS_data_africa/flair/y/images/"+i+"_"+ str(s), image)
    # image = squarify(image)


    lower_bound, upper_bound = np.percentile(image, 0.5), np.percentile(image, 99.5)
    image_data_pre = np.clip(image, lower_bound, upper_bound)
    d = (np.max(image_data_pre)-np.min(image_data_pre))
    d = 1 if d==0 else d
    image_data_pre = ((image_data_pre - np.min(image_data_pre))/d) *255.0
    image_data_pre[image==0] = 0
    image = transform.resize(image_data_pre[:,:], (image_size, image_size), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
    image = np.stack((image, image, image), axis = -1 )
    image = np.uint8(image)

    sam_transform = ResizeLongestSide(sam_model.image_encoder.img_size)
    image = sam_transform.apply_image(image)
    # resized_shapes.append(resize_img.shape[:2])
    image = torch.as_tensor(image.transpose(2, 0, 1)).to('cuda')
    # model input: (1, 3, 1024, 1024)
    image = sam_model.preprocess(image[None,:,:,:]) # (1, 3, 1024, 1024)
    image = image[0,...] # (3, 1024, 1024)
    # plt.imshow(image[1,:,:].cpu(), cmap ="gray")
    # plt.show()
    with torch.no_grad():
      embedding = sam_model.image_encoder(image[None,...])[0].cpu().numpy()
      # print(embedding.shape)
    np.save(base + "BraTS_data/t2/z/embeddings/" +i+ "_"+ str(s), embedding)

100%|██████████| 1/1 [05:10<00:00, 310.06s/it]


## DATA

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=QpiZpPvpBXGH8aJzKXToTdveIwrVmD&prompt=consent&access_type=offline&code_challenge=aSKygISn9ScqtikJ49pwpChLLwzPevsVsuy_L210CY8&code_challenge_method=S256

Enter authorization code: 4/0Adeu5BUt0fB4wwfHaOOwQ8dc6x9J-PMkd4iitRq_wdhUL_aCDEi6JtnWfCXEFl5tlXOtkg

You are now logged in as [hannod98@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
!sudo zypper install gcc python3-devel
!sudo pip uninstall crcmod
!sudo pip install --no-cache-dir -U crcmod

sudo: zypper: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=31406 sha256=3c59668cc183833fbb6ffb3d1b7ad9b2f8e8dcd3a251e685862cd41d6e059c18
  Stored in directory: /tmp/pip-ephem-wheel-cache-pjndf8_a/wheels/85/4c/07/72215c529bd59d67e3dac29711d7aba1b692f543c808ba9e86
Successfully built crcmod


In [ ]:
!mkdir BraTS_data_africa

In [ ]:
!gcloud config set project 	crafty-nova-394910
!gsutil -m cp -r gs://embedding_spark/BraTS_data_africa/t1/t1 ./BraTS_data_africa/
!gsutil -m cp -r gs://embedding_spark/BraTS_data_africa/t2 ./BraTS_data_africa/
!gsutil -m cp -r gs://embedding_spark/BraTS_data_africa/flair ./BraTS_data_africa/

Streaming output truncated to the last 5000 lines.
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_65.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_66.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_67.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_68.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_69.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_7.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_70.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_71.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_72.npy...
Copying gs://embedding_spark/BraTS_data_africa/flair/z/embeddings/BraTS-SSA-00028-000_

# Prepare

In [ ]:
! pip install monai
! pip install pytorch-lightning
! pip install torchio

In [ ]:
from monai.networks.nets import UNet
from torchsummary import summary
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset
from tqdm import tqdm
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
import random
import pytorch_lightning as pl
import torchio as tio
from pytorch_lightning.callbacks import ModelCheckpoint



In [ ]:
def get_bounding_box(mask):

  y_indices, x_indices = np.where(mask > 0)
  if len(y_indices) == 0:
    return [0,0,0,0]
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = mask.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  # bboxes = np.array([x_min, y_min, x_max, y_max])

  return [x_min, y_min, x_max, y_max]




def get_3d_from_SAM(embeddings_path, mask, case_id, dimention, modality, sam_model, decoder,device):
  sam_model#.to(device)

  output = np.zeros_like(mask)

  if dimention == "z":
    sclices = mask.shape[2]
    for s in range(sclices):
      emb =  np.load(embeddings_path + "/" + modality + "/" + dimention + "/embeddings/"+ case_id + "_" + str(s) +".npy" )
      emb = torch.as_tensor(emb, dtype=torch.float)
      temp_mask = mask[:,:,s]

      box = get_bounding_box(temp_mask)
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (temp_mask.shape[-2], temp_mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float)

      with torch.no_grad():
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(points=None, boxes=box_torch, masks=None)


      with torch.no_grad():
        mask_predictions, _ = decoder(
              image_embeddings= emb[None,...], # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

      masks = sam_model.postprocess_masks(mask_predictions, (1024, 1024), (240, 240))
      masks = masks > sam_model.mask_threshold
      masks = np.squeeze(masks.numpy(), axis = 0)
      masks = np.int32(masks)
      masks = np.squeeze(masks, axis = 0)

      output[:,:,s] = masks


  if dimention == "y":
    sclices = mask.shape[1]
    for s in range(sclices):
      emb =  np.load(embeddings_path + "/" + modality + "/" + dimention + "/embeddings/"+ case_id + "_" + str(s) +".npy" )
      emb = torch.as_tensor(emb, dtype=torch.float)
      temp_mask = mask[:,s,:]

      box = get_bounding_box(temp_mask)
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (temp_mask.shape[-2], temp_mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float)

      with torch.no_grad():
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(points=None, boxes=box_torch, masks=None)


      with torch.no_grad():
        mask_predictions, _ = decoder(
              image_embeddings= emb[None,...], # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

      masks = sam_model.postprocess_masks(mask_predictions, (1024, 1024), (240, 240))
      masks = masks > sam_model.mask_threshold
      masks = np.squeeze(masks.numpy(), axis = 0)
      masks = np.int32(masks)
      masks = np.squeeze(masks, axis = 0)

      output[:,s,:] = masks


  if dimention == "x":
    sclices = mask.shape[0]
    for s in range(sclices):
      emb =  np.load(embeddings_path + "/" + modality + "/" + dimention + "/embeddings/"+ case_id + "_" + str(s) +".npy" )
      emb = torch.as_tensor(emb, dtype=torch.float)
      temp_mask = mask[s,:,:]

      box = get_bounding_box(temp_mask)
      box_np = np.array(box)
      sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
      box = sam_trans.apply_boxes(box_np, (temp_mask.shape[-2], temp_mask.shape[-1]))
      box_torch = torch.as_tensor(box, dtype=torch.float)

      with torch.no_grad():
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(points=None, boxes=box_torch, masks=None)

      with torch.no_grad():
        mask_predictions, _ = decoder(
              image_embeddings= emb[None,...], # (B, 256, 64, 64)
              image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
              sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
              dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
              multimask_output=False,
            )

      masks = sam_model.postprocess_masks(mask_predictions, (1024, 1024), (240, 240))
      masks = masks > sam_model.mask_threshold
      masks = np.squeeze(masks.numpy(), axis = 0)
      masks = np.int32(masks)
      masks = np.squeeze(masks, axis = 0)

      output[s,:,:] = masks

  return output



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
sam_model = sam_model_registry[model_type](checkpoint=checkpoint)#.to('cuda')


decoder = sam_model.mask_decoder
model_path = "/content/sam_model_best_more_epochs.pth"
checkpoint = torch.load(model_path, map_location="cpu")
decoder.load_state_dict(checkpoint)
# decoder

<All keys matched successfully>

In [ ]:
embedding_path = "/content/BraTS_data_africa"
ground_truth_path = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData"

class BraTSDataset(Dataset):

    def __init__(self, ground_truth_path, embedding_path, sam, decoder, device, t = True):
        """
        Args:
            embedding_path (string): Path to  images.
            masks_path (string): path to masks.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ground_truth_path = ground_truth_path
        self.device = device
        self.embedding_path = embedding_path
        if t:
            self.data = os.listdir(self.ground_truth_path)[:40]
        else:
            self.data = os.listdir(self.ground_truth_path)[40:]
        self.sam = sam
        self.decoder = decoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):


      t2f =  nib.load(self.ground_truth_path + "/" + self.data[idx] + "/"+ self.data[idx] +"-t2f.nii.gz")
      t2f = t2f.get_fdata()
      lower_bound, upper_bound = np.percentile(t2f, 0.5), np.percentile(t2f, 99.5)
      image_data_pre = np.clip(t2f, lower_bound, upper_bound)
      d = (np.max(image_data_pre)-np.min(image_data_pre))
      d = 1 if d==0 else d
      image_data_pre = ((image_data_pre - np.min(image_data_pre))/d)
      image_data_pre[t2f==0] = 0
      t2f = image_data_pre


      t1n =  nib.load(self.ground_truth_path + "/" + self.data[idx] + "/"+ self.data[idx] +"-t1n.nii.gz")
      t1n = t1n.get_fdata()
      lower_bound, upper_bound = np.percentile(t1n, 0.5), np.percentile(t1n, 99.5)
      image_data_pre = np.clip(t1n, lower_bound, upper_bound)
      d = (np.max(image_data_pre)-np.min(image_data_pre))
      d = 1 if d==0 else d
      image_data_pre = ((image_data_pre - np.min(image_data_pre))/d)
      image_data_pre[t1n==0] = 0
      t1n = image_data_pre


      t2w =  nib.load(self.ground_truth_path + "/" + self.data[idx] + "/"+ self.data[idx] +"-t2w.nii.gz")
      t2w = t2w.get_fdata()
      lower_bound, upper_bound = np.percentile(t2w, 0.5), np.percentile(t2w, 99.5)
      image_data_pre = np.clip(t2w, lower_bound, upper_bound)
      d = (np.max(image_data_pre)-np.min(image_data_pre))
      d = 1 if d==0 else d
      image_data_pre = ((image_data_pre - np.min(image_data_pre))/d)
      image_data_pre[t2w==0] = 0
      t2w = image_data_pre


      t1c =  nib.load(self.ground_truth_path + "/" + self.data[idx] + "/"+ self.data[idx] +"-t1c.nii.gz")
      t1c = t1c.get_fdata()
      lower_bound, upper_bound = np.percentile(t1c, 0.5), np.percentile(t1c, 99.5)
      image_data_pre = np.clip(t1c, lower_bound, upper_bound)
      d = (np.max(image_data_pre)-np.min(image_data_pre))
      d = 1 if d==0 else d
      image_data_pre = ((image_data_pre - np.min(image_data_pre))/d)
      image_data_pre[t1c==0] = 0
      t1c = image_data_pre

      #-------------------------------------------------------------------------------------#

      mask =  nib.load(self.ground_truth_path + "/" + self.data[idx] + "/"+ self.data[idx] +"-seg.nii.gz")
      mask = mask.get_fdata()

      #-------------------------------------------------------------------------------------#

      # t1_x_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "x", "t1", self.sam , self.decoder, self.device)
      # t1_y_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "y", "t1", self.sam ,self.decoder, self.device)
      t1_z_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "z", "t1", self.sam ,self.decoder, self.device)

      # t2_x_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "x", "t2", self.sam ,self.decoder, self.device)
      # t2_y_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "y", "t2", self.sam ,self.decoder, self.device)
      t2_z_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "z", "t2", self.sam ,self.decoder, self.device)

      # t1ce_x_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "x", "t1ce", self.sam ,self.decoder, self.device)
      # t1ce_y_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "y", "t1ce", self.sam ,self.decoder, self.device)
      # t1ce_z_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "z", "t1ce", self.sam ,self.decoder, self.device)

      # flair_x_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "x", "flair", self.sam ,self.decoder, self.device)
      # flair_y_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "y", "flair", self.sam ,self.decoder, self.device)
      flair_z_mask = get_3d_from_SAM(self.embedding_path, mask, self.data[idx], "z", "flair", self.sam ,self.decoder, self.device)
      #-------------------------------------------------------------------------------------#



      # input_ = np.stack((t1_x_mask, t1_y_mask, t1_z_mask, t2_x_mask, t2_y_mask, t2_z_mask, t1ce_x_mask, t1ce_y_mask,
      #                   t1ce_z_mask, flair_x_mask, flair_y_mask, flair_z_mask, t2f, t1n, t2w, t1c), axis = 0)


      input_ = np.stack(( t2f, t1n, t2w, t1c, t1_z_mask, t2_z_mask, flair_z_mask), axis = 0)

      input_ = np.float32(input_)
      input_ = torch.as_tensor(input_)
      extra = np.zeros((240, 240, 1))
      mask = np.concatenate((mask, extra), axis = -1)

      class_1 = np.zeros_like(mask)
      class_2 = np.zeros_like(mask)
      class_3 = np.zeros_like(mask)

      class_1[np.where(mask == 1)] = 1
      class_2[np.where(mask == 2)] = 1
      class_3[np.where(mask == 3)] = 1

      # 1 is core 2 is edema 3 is enhancing
      # class_1[np.where(mask == 1)] = 1
      # class_1[np.where(mask == 2)] = 1
      # class_1[np.where(mask == 3)] = 1

      # class_2[np.where(mask == 1)] = 1
      # class_2[np.where(mask == 3)] = 1

      # class_3[np.where(mask == 1)] =  1

      mask = mask = np.stack((class_1, class_2, class_3), axis = 0)


      mask = torch.as_tensor(mask)


      sample = {"input": input_,
                  'mask': mask,
                  }

      return sample

In [ ]:
from google.cloud import storage


def write_read(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Mode can be specified as wb/rb for bytes mode.
    # See: https://docs.python.org/3/library/io.html
    with blob.open("w") as f:
        f.write("Hello world")

In [ ]:
# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "voting_cash"

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)

print(f"Bucket {bucket.name} created.")

RefreshError: ignored

In [ ]:
train = BraTSDataset(ground_truth_path, embedding_path, sam_model, decoder, device)

for i in range(len(train)):
  inp = train[i]["input"]
  out = train[i]["mask"]
  print(train[i])
  break

{'input': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ...

In [ ]:
train = BraTSDataset(ground_truth_path, embedding_path, sam_model, decoder, device)
val = BraTSDataset(ground_truth_path, embedding_path, sam_model, decoder, device, t = False)

print(len(train))
print(len(val))

train_loader = DataLoader(train, batch_size= 8, shuffle=True, num_workers=2, prefetch_factor= 2)
val_loader = DataLoader(val, batch_size= 8, shuffle=False, num_workers=2, prefetch_factor= 2)



hprams = {
    "num_epochs": 200,
    "best_loss": 1e10,
    "model_save_path": "/content/drive/MyDrive/miccai_data",
    "lr":1e-3,
    "weight_decay":0,
    "device": device,
    "train_dataloader": train_loader,
    "val_dataloader": val_loader
    }

40
3


In [ ]:
net = UNet(
    spatial_dims=3,
    in_channels=7,
    out_channels=3,
    channels=(16, 32, 64),
    strides=(2, 2),
).to(device)

In [ ]:
summary(net, (7, 240, 240, 155))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 16, 120, 120, 78]           3,040
    InstanceNorm3d-2     [-1, 16, 120, 120, 78]               0
           Dropout-3     [-1, 16, 120, 120, 78]               0
             PReLU-4     [-1, 16, 120, 120, 78]               1
            Conv3d-5       [-1, 32, 60, 60, 39]          13,856
    InstanceNorm3d-6       [-1, 32, 60, 60, 39]               0
           Dropout-7       [-1, 32, 60, 60, 39]               0
             PReLU-8       [-1, 32, 60, 60, 39]               1
            Conv3d-9       [-1, 64, 60, 60, 39]          55,360
   InstanceNorm3d-10       [-1, 64, 60, 60, 39]               0
          Dropout-11       [-1, 64, 60, 60, 39]               0
            PReLU-12       [-1, 64, 60, 60, 39]               1
   SkipConnection-13       [-1, 96, 60, 60, 39]               0
  ConvTranspose3d-14     [-1, 16, 120, 

In [ ]:
s = np.zeros((1,7, 240, 240, 155), dtype= np.float32)
s = torch.from_numpy(s).to("cuda")
net(s).shape

torch.Size([1, 3, 240, 240, 156])

In [ ]:
loss_function = monai.losses.DiceLoss(sigmoid=True, to_onehot_y = False).to(device)
optimizer = torch.optim.Adam(net.parameters(), hprams["lr"])

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class

    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer

    def prepare_batch(self, batch):
        return (batch["input"], batch["mask"])

    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

In [ ]:
model = Model(
    net=net,
    criterion=loss_function,
    learning_rate=hprams["lr"],
    optimizer_class=torch.optim.Adam,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor="val_loss",)

checkpoint_callback = ModelCheckpoint(
    dirpath = hprams["model_save_path"],
    filename = "voting_unet_small",
    save_last = True,
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
)

trainer = pl.Trainer(
    default_root_dir= hprams["model_save_path"],
    callbacks=[early_stopping, checkpoint_callback],
    log_every_n_steps=3,
    max_epochs = hprams["num_epochs"],
    # accelerator="gpu", devices=2, num_nodes=2, strategy="ddp"
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
for x in train_loader:
  pass

KeyboardInterrupt: ignored

In [ ]:
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type     | Params
---------------------------------------
0 | net       | UNet     | 116 K 
1 | criterion | DiceLoss | 0     
---------------------------------------
116 K     Trainable params
0         Non-trainable params
116 K     Total params
0.465     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

RuntimeError: ignored

In [ ]:
trainer.save_checkpoint("example.ckpt")

In [ ]:
torch.save({
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, "/content/drive/MyDrive/save_to/voting/model.pt")

In [ ]:
checkpoint = torch.load("/content/model.pt")
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


# 3 ch to 1 ch

In [ ]:
def tranform_mask(mask):
  res = np.zeros(mask.shape[1:])

  msk1 = mask[0]
  msk2 = mask[1]
  msk3 = mask[2]

  res[msk1>0] = 1
  res[msk2>0] = 2
  res[msk3>0] = 3

  return res


